In [1]:
import requests
from urllib.parse import urlencode
from urllib.request import urlopen 
from bs4 import BeautifulSoup 
import re
import json
import csv
from urllib.parse import quote
from urllib.parse import quote_plus
from urllib.parse import urlsplit
from urllib.parse import urlunsplit
import urllib.error
import pandas as pd
from geopy.distance import geodesic 

In [2]:
api_key = ''

In [3]:
data = pd.read_csv (r'C:\Users\timohakartoha\SM Project\test2.csv')
data.head(5)

,Store name,Store Category1,Store Category2,Store Category3,Address,City,Province,Postal Code,Latitude,Longitude
0,1-800-GOT-JUNK in Surrey,/store/category/Construction,/store/category/Moving,NaN,12353 104 Avenue\r\n Suite #6,Surrey,British Columbia,V3V 3H3,49.193698,-122.881641
1,1-800-GOT-JUNK in Vancouver,/store/category/Construction,/store/category/Moving,NaN,8410 Ontario St\r\n Suite #102,Vancouver,British Columbia,V5X 4S6,49.207518,-123.106240
2,1-800-GOT-JUNK in Winnipeg,/store/category/Construction,/store/category/Moving,NaN,3000 McGillivray Boulevard\n,Winnipeg,Manitoba,R3Y 0A1,49.809500,-97.229766
3,1-800-GOT-JUNK in London,/store/category/Construction,/store/category/Moving,NaN,1025 Hargrieve Road\r\n Suite 1,London,Ontario,N6E 1P7,42.931311,-81.216521
4,1-800-GOT-JUNK in Ottawa,/store/category/Construction,/store/category/Moving,NaN,2473 Sheffield Road\n,Ottawa,Ontario,K1B 3V6,45.409778,-75.608416


In [4]:
population = pd.read_csv (r'C:\Users\timohakartoha\SM Project\supp_db2.csv')
population.head(5)

,postcode,population
0,A0A0C2,2
1,A0A1A0,61
2,A0A1B0,480
3,A0A1C0,1091
4,A0A1E0,315


In [5]:
data.rename({"Store name":"StoreName"}, axis="columns", inplace=True)
data.rename({"Postal Code":"PostalCode"}, axis="columns", inplace=True)
data.rename({"Store Category1":"StoreCategory1"}, axis="columns", inplace=True)
data.rename({"Store Category2":"StoreCategory2"}, axis="columns", inplace=True)
data.rename({"Store Category3":"StoreCategory3"}, axis="columns", inplace=True)

In [6]:
data['Category1'] = data['StoreCategory1'].str.split('category/').str[1]
data['Category2'] = data['StoreCategory2'].str.split('category/').str[1]
data['Category3'] = data['StoreCategory3'].str.split('category/').str[1]

In [7]:
data = data.drop(['StoreCategory1', 'StoreCategory2', 'StoreCategory3'], axis=1)

In [8]:
data.Category1 = data.Category1.str.replace('+', ', ')
data.Category2 = data.Category2.str.replace('+', ', ')
data.Category2 = data.Category3.str.replace('+', ', ')
data.Address = data.Address.str.replace('\n', '')
data.PostalCode = data.PostalCode.str.replace(' ', '')
data.PostalCode = data.PostalCode.str.replace('-', '')




In [9]:
data.head(5)

,StoreName,Address,City,Province,PostalCode,Latitude,Longitude,Category1,Category2,Category3
0,1-800-GOT-JUNK in Surrey,12353 104 Avenue\r Suite #6,Surrey,British Columbia,V3V3H3,49.193698,-122.881641,Construction,NaN,NaN
1,1-800-GOT-JUNK in Vancouver,8410 Ontario St\r Suite #102,Vancouver,British Columbia,V5X4S6,49.207518,-123.106240,Construction,NaN,NaN
2,1-800-GOT-JUNK in Winnipeg,3000 McGillivray Boulevard,Winnipeg,Manitoba,R3Y0A1,49.809500,-97.229766,Construction,NaN,NaN
3,1-800-GOT-JUNK in London,1025 Hargrieve Road\r Suite 1,London,Ontario,N6E1P7,42.931311,-81.216521,Construction,NaN,NaN
4,1-800-GOT-JUNK in Ottawa,2473 Sheffield Road,Ottawa,Ontario,K1B3V6,45.409778,-75.608416,Construction,NaN,NaN


In [15]:
def find_competitors(potential_location):
    endpoint = f'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': potential_location, 'key': api_key}
    url_params = urlencode(params)
    url = f'{endpoint}?{url_params}'
    r = requests.get(url)
    latlng = {}
    try:
        latlng = r.json()['results'][0]['geometry']['location']
    except:
        pass
    potential_location_latlng = (latlng.get('lat'), latlng.get('lng'))
    
    
    data['Distance'] = data.apply(lambda x:geodesic((x["Latitude"], x["Longitude"]), potential_location_latlng), axis=1)
    
    
    Radius_2km = data.loc[data['Distance'] < 2.00]
    Competitiors_in_2km = Radius_2km['StoreName'].count()
    
    Radius_5km = data.loc[(data['Distance'] >= 2.00) & (data['Distance'] < 5.00)]
    Competitiors_in_5km = Radius_5km['StoreName'].count()
    Radius_10km = data.loc[(data['Distance'] >= 5.00) & (data['Distance'] <= 10.00)]
    Competitiors_in_10km = Radius_10km['StoreName'].count()
    
    print("Population in potential location:")
    print(population[population.postcode == potential_location].to_string(index=False))
    print('___________________________________________________________________________')
    
    print("Number of competitors in 2 km radius:")
    print(Competitiors_in_2km)
    
    print("Stores in radius 2 km")
    print(Radius_2km[['StoreName', 'City', 'Category1', 'Category2']].to_string(index=False))
    print('___________________________________________________________________________')
    
    print("Number of competitors in 5 km radius:")
    print(Competitiors_in_5km)

    print("Stores in radius 5 km")
    print(Radius_5km[['StoreName', 'City', 'Category1', 'Category2']].to_string(index=False))
    print('___________________________________________________________________________')
    
    print("Number of competitors in 10 km radius:")
    print(Competitiors_in_10km)

    print("Stores in radius 10 km")
    print(Radius_10km[['StoreName', 'City', 'Category1', 'Category2']].to_string(index=False))
    
    
    

In [16]:
find_competitors('V5K1C4')

Population in potential location:
postcode  population
  V5K1C4          33
___________________________________________________________________________
Number of competitors in 2 km radius:
107
Stores in radius 2 km
                                             StoreName             City          Category1             Category2
                     Atlas Snowshoe in North Vancouver  North Vancouver            Outdoor                   NaN
                     Atlas Snowshoe in North Vancouver  North Vancouver            Outdoor                   NaN
                                  Biolite in Vancouver        Vancouver            Outdoor               Cooking
                         Cannondale in NORTH VANCOUVER  NORTH VANCOUVER               Bike                   NaN
                      Carve Designs in North Vancouver  North Vancouver           Clothing                   NaN
                         Church's Chicken in Vancouver        Vancouver         Restaurant                